In [4]:
import pandas as pd
from pytube import YouTube, Playlist
import pydub
import os
import re
import glob

os.environ["IMAGEIO_FFMPEG_EXE"] = "/opt/homebrew/bin/ffmpeg"

import moviepy.editor as mp

target_dir = "./all_data"

In [5]:
def mp4_to_mp3(in_path, out_path):
    audio = mp.AudioFileClip(in_path)
    audio.write_audiofile(out_path)

In [6]:


if not os.path.isdir(target_dir + "/mp4"):
    os.makedirs(target_dir + "/mp4")
if not os.path.isdir(target_dir + "/mp3"):
    os.makedirs(target_dir + "/mp3")



traincsv = open(target_dir + "/train.csv", "w")
traincsv.write("\"file\",\"Description\"\n")
failfile = open(target_dir + "/fail.txt", "w")
list_url = "https://www.youtube.com/playlist?list=PL96kIIcXJpMtmsQGlsNVqWduASZnh4HnE"
playlist = Playlist(list_url)

idx = 0
for video in playlist.videos:

    
    mp4_path = target_dir + "/mp4/" + str(idx) + ".mp4"
    mp3_path = target_dir + "/mp3/" + str(idx) + ".mp3"

    if not os.path.isfile(mp4_path):
        try:
            video.streams.filter().get_highest_resolution().download(filename = str(idx) + ".mp4", output_path=target_dir + "/mp4")
        except:
            print(idx, video.title, "cant downlaod")
            failfile.write(str(idx) + ". " + "cant downlaod.\n")
            continue
    
    if not os.path.isfile(mp3_path):
        try:
            mp4_to_mp3(mp4_path, mp3_path)
        except:
            print(idx, "cant convert")
            failfile.write(str(idx) + ". " + "cant convert.\n")
            continue

    print(idx, "success")
    idx += 1

traincsv.close()
failfile.close()
    



MoviePy - Writing audio in ./all_data/mp3/0.mp3


MoviePy - Done.
0 success
MoviePy - Writing audio in ./all_data/mp3/1.mp3


1 cant convert


MoviePy - Writing audio in ./all_data/mp3/1.mp3


MoviePy - Done.
1 success
MoviePy - Writing audio in ./all_data/mp3/2.mp3


MoviePy - Done.
2 success
